In [1]:
import pandas as pd
import csv
import os
import time
from collections import defaultdict
import numpy as np
import itertools
import db_structure
import db_structure_v2
import networkx as nx
from imp import reload
import matplotlib.pyplot as plt
import utilities as u
from flask import jsonify
import logging
# import exceptions
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

logging.basicConfig(format=' %(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
%load_ext autoreload
%aimport -web
%autoreload 2

In [ ]:
draw_db = db

G = nx.DiGraph()
G.add_nodes_from(draw_db.table_names)
for table_name in draw_db.table_names:
    for sibling in draw_db.find_table_siblings(table_name):
        G.add_edge(table_name, sibling)
        G.add_edge(sibling, table_name)
    for child in draw_db.find_table_children(table_name):
        G.add_edge(table_name, child)
plt.figure(figsize=(10,10))

#draw with networkx built-ins
nx.draw_networkx(G, node_shape="None", width=0.2)

#OR

#draw using pygraphviz
#A = nx.nx_agraph.to_agraph(G)
#H = nx.nx_agraph.from_agraph(A)
#nx.draw_spring(H, node_shape="None", with_labels=True, width=0.2)

In [68]:
x = db_structure_v2.DBMaker(dataset_name='TOPICC', directory_path='datasets\\TOPICC')

In [70]:
x.create_db()

Exception: datasets\TOPICC is already in the db

In [ ]:
y = db_structure_v2.DBLinker(directory_path='datasets\\TOPICC')

In [ ]:
y.add_global_fk('PudID')

In [ ]:
y.finalize()

In [ ]:
z = db_structure_v2.DBExtractor(directory_path='datasets\\TOPICC')

In [ ]:
z.find_paths_between_tables('CPR', 'CARDIACPROCEDURES')

In [ ]:
z.find_paths_multi_tables(['HOSPITALADMIT', 'DEATH', 'CPR'])

In [ ]:
path = ['HOSPITALADMIT', 'CAREPROCESSES', 'DEATH']
start = time.time()
df = z.get_df_from_path(path, table_columns_of_interest=[('HOSPITALADMIT', 'PudID'), ('HOSPITALADMIT', 'Sex'), ('CAREPROCESSES', 'MechVent'), ('DEATH', 'DeathMode')])
end = time.time()
print(end-start)

In [ ]:
df

In [ ]:
df['DeathMode'].value_counts()

In [ ]:
filters = {
    'AdmitThroughED': {'type': 'list', 'filter': ['Yes', 'No']},
    'Sex': {'type': 'list', 'filter': ['Male', 'Female']}
}
new_df = z.aggregate_df(df, groupby_columns=['AdmitThroughED', 'Sex'], filters=filters)

In [ ]:
sorted(new_df.loc[:,'groupby_labels'].unique(), key=lambda x: x.upper())

In [ ]:
x, y = pd.cut(new_df['Count'], bins=2)

In [ ]:
for i in u.pairwise([1,2]):
    print(i)

In [ ]:
'3'.split('.')

In [ ]:
str((5, 3))

In [ ]:
path = ['HOSPITALADMIT', 'CAREPROCESSES', 'PHYSIOSTATUS']
df = z.get_df_from_path(path, table_columns_of_interest=['HOSPITALADMIT.Sex', 'CAREPROCESSES.MechVent', 'PHYSIOSTATUS.LowpH'])

In [ ]:
filters = {
    'MechVent': {'type': 'list', 'filter': ['Yes']},
    'LowpH': {'type': 'range', 'filter': {'min': 6.8, 'max': 6.9, 'bins': 4}},
    'Sex': None
}

z.aggregate_df(df, groupby_columns=['MechVent', 'LowpH'], filters=filters, aggregate_column='Sex')

In [ ]:
len(f)

In [ ]:
f.sort_values(by='LowpH')

In [ ]:
w = db_structure_v2.DBCustomizer(directory_path='datasets\\TOPICC')

In [ ]:
w.rename_column('CARDIACPROCEDURES', 'PudID', 'PatientID')

In [ ]:
w.dump_customization()

In [ ]:
x = [(1, 2), (3, 4)]
for i, j in x:
    print(i)
    print(j)

In [9]:
from web import db

In [67]:
db.session.rollback()

In [ ]:
db.engine